# CS579: Lecture 13  

**Demographic Inference II**

*[Dr. Aron Culotta](http://cs.iit.edu/~culotta)*  
*[Illinois Institute of Technology](http://iit.edu)*

## Gender Classification

Let's build a classifier to predict whether a Twitter user is male/female.

We'll collect "labeled" training data using Census name list.

**1.) Collect Census names. **

In [1]:
# Fetch male/female names from Census.

import requests

def get_census_names():
    """ Fetch a list of common male/female names from the census.
    For ambiguous names, we select the more frequent gender."""
    males = requests.get('http://www2.census.gov/topics/genealogy/1990surnames/dist.male.first').text.split('\n')
    females = requests.get('http://www2.census.gov/topics/genealogy/1990surnames/dist.female.first').text.split('\n')
    males_pct = dict([(m.split()[0].lower(), float(m.split()[1]))
                  for m in males if m])
    females_pct = dict([(f.split()[0].lower(), float(f.split()[1]))
                    for f in females if f])
    male_names = set([m for m in males_pct if m not in females_pct or
                  males_pct[m] > females_pct[m]])
    female_names = set([f for f in females_pct if f not in males_pct or
                  females_pct[f] > males_pct[f]])    
    return male_names, female_names

male_names, female_names = get_census_names()
print('found %d female and %d male names' % (len(male_names), len(female_names)))
print('male name sample:', list(male_names)[:5])
print('female name sample:', list(female_names)[:5])

found 1146 female and 4014 male names
male name sample: ['harry', 'ruben', 'jamal', 'francisco', 'foster']
female name sample: ['tyra', 'benita', 'arnetta', 'williemae', 'oralee']


**2.) Sample 5K tweets with names on the Census list. **

In [2]:
# Construct TwitterAPI object.

import configparser
from TwitterAPI import TwitterAPI

def get_twitter(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')

In [5]:
# Sample U.S. tweets with names from Census. 
import sys

def get_first_name(tweet):
    if 'user' in tweet and 'name' in tweet['user']:
        parts = tweet['user']['name'].split()
        if len(parts) > 0:
            return parts[0].lower()

def sample_tweets(twitter, limit, male_names, female_names):
    tweets = []
    while True:
        try:
            # Restrict to U.S.
            for response in twitter.request('statuses/filter',
                        {'locations':'-124.637,24.548,-66.993,48.9974'}):
                if 'user' in response:
                    name = get_first_name(response)
                    if name in male_names or name in female_names:
                        tweets.append(response)
                        if len(tweets) % 100 == 0:
                            print('found %d tweets' % len(tweets))
                        if len(tweets) >= limit:
                            return tweets
        except:
            print("Unexpected error:", sys.exc_info()[0])
    return tweets
        
tweets = sample_tweets(twitter, 5000, male_names, female_names)

In [3]:
# optionally read from disk
# import pickle
# tweets = pickle.load(open('tweets.pkl', 'rb'))

In [6]:
from collections import Counter

print('sampled %d tweets' % len(tweets))
print('top names:', Counter(get_first_name(t) for t in tweets).most_common(10))

sampled 5000 tweets
top names: [('michelle', 58), ('john', 54), ('david', 50), ('chris', 48), ('michael', 48), ('mike', 46), ('james', 41), ('lisa', 38), ('alex', 36), ('daniel', 33)]


In [5]:
# Save these tweets.
import pickle
pickle.dump(tweets, open('tweets.pkl', 'wb'))

**3.) Tokenize tweets. **

In [7]:
test_tweet = tweets[200]
print('test tweet:\n\tscreen_name=%s\n\tname=%s\n\tdescr=%s\n\ttext=%s' %
      (test_tweet['user']['screen_name'],
       test_tweet['user']['name'],
       test_tweet['user']['description'],
       test_tweet['text']))

test tweet:
	screen_name=mrchadhurst
	name=Chad
	descr=Tweeting about sports, politics, and other trending topics.
	text=This is a big case for the tech world! I am siding with @Microsoft on this one. It's time for legislative reform https://t.co/0IufAp80KR


In [9]:
import re

def tokenize(string, lowercase, keep_punctuation, prefix,
             collapse_urls, collapse_mentions):
    """ Split a tweet into tokens."""
    if not string:
        return []
    if lowercase:
        string = string.lower()
    tokens = []
    if collapse_urls:
        string = re.sub('http\S+', 'THIS_IS_A_URL', string)
    if collapse_mentions:
        string = re.sub('@\S+', 'THIS_IS_A_MENTION', string)
    if keep_punctuation:
        tokens = string.split()
    else:
        tokens = re.sub('\W+', ' ', string).split()
    if prefix:
        tokens = ['%s%s' % (prefix, t) for t in tokens]
    return tokens

In [10]:
tokenize(test_tweet['user']['description'], lowercase=True,
         keep_punctuation=True, prefix='d=',
         collapse_urls=True, collapse_mentions=True)

['d=tweeting',
 'd=about',
 'd=sports,',
 'd=politics,',
 'd=and',
 'd=other',
 'd=trending',
 'd=topics.']

In [11]:
tokenize(test_tweet['text'], lowercase=True, keep_punctuation=True,
         prefix='t=',
         collapse_urls=True, collapse_mentions=False)

['t=this',
 't=is',
 't=a',
 't=big',
 't=case',
 't=for',
 't=the',
 't=tech',
 't=world!',
 't=i',
 't=am',
 't=siding',
 't=with',
 't=@microsoft',
 't=on',
 't=this',
 't=one.',
 "t=it's",
 't=time',
 't=for',
 't=legislative',
 't=reform',
 't=THIS_IS_A_URL']

In [12]:
def tweet2tokens(tweet, use_descr=True, lowercase=True,
                 keep_punctuation=True, descr_prefix='d=',
                 collapse_urls=True, collapse_mentions=True):
    """ Convert a tweet into a list of tokens, from the tweet text and optionally the
    user description. """
    tokens = tokenize(tweet['text'], lowercase, keep_punctuation, None,
                       collapse_urls, collapse_mentions)
    if use_descr:
        tokens.extend(tokenize(tweet['user']['description'], lowercase,
                               keep_punctuation, descr_prefix,
                               collapse_urls, collapse_mentions))
    return tokens

In [13]:
# for enumerating all possible arguments of tweet2tokens
# https://docs.python.org/2/library/itertools.html#itertools.product
from itertools import product

use_descr_opts = [True, False]
lowercase_opts = [True, False]
keep_punctuation_opts = [True, False]
descr_prefix_opts = ['d=', '']
url_opts = [True, False]
mention_opts = [True, False]

argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention']
option_iter = product(use_descr_opts, lowercase_opts,
                       keep_punctuation_opts,
                       descr_prefix_opts, url_opts,
                       mention_opts)
for options in option_iter:
    print('  '.join('%s=%s' % (name, opt) for name, opt in zip(argnames, options)))
    print
    print('  '.join(tweet2tokens(test_tweet, *options)), '\n----\n')

use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=True
this  is  a  big  case  for  the  tech  world!  i  am  siding  with  THIS_IS_A_MENTION  on  this  one.  it's  time  for  legislative  reform  THIS_IS_A_URL  d=tweeting  d=about  d=sports,  d=politics,  d=and  d=other  d=trending  d=topics. 
----

use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=False
this  is  a  big  case  for  the  tech  world!  i  am  siding  with  @microsoft  on  this  one.  it's  time  for  legislative  reform  THIS_IS_A_URL  d=tweeting  d=about  d=sports,  d=politics,  d=and  d=other  d=trending  d=topics. 
----

use_descr=True  lower=True  punct=True  prefix=d=  url=False  mention=True
this  is  a  big  case  for  the  tech  world!  i  am  siding  with  THIS_IS_A_MENTION  on  this  one.  it's  time  for  legislative  reform  https://t.co/0iufap80kr  d=tweeting  d=about  d=sports,  d=politics,  d=and  d=other  d=trending  d=topics. 
----

use_descr=True  lower=True  punc

In [58]:
# Let's tokenize all tweets.
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=False, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True)
              for t in tweets]

In [59]:
tokens_list[200]

['this',
 'is',
 'a',
 'big',
 'case',
 'for',
 'the',
 'tech',
 'world',
 'i',
 'am',
 'siding',
 'with',
 'THIS_IS_A_MENTION',
 'on',
 'this',
 'one',
 'it',
 's',
 'time',
 'for',
 'legislative',
 'reform',
 'THIS_IS_A_URL',
 'd=tweeting',
 'd=about',
 'd=sports',
 'd=politics',
 'd=and',
 'd=other',
 'd=trending',
 'd=topics']

In [60]:
# Store these in a sparse matrix.

#1) Create a vocabulary (dict from term->index)

# https://docs.python.org/2/library/collections.html#collections.defaultdict
from collections import defaultdict

def make_vocabulary(tokens_list):
    vocabulary = defaultdict(lambda: len(vocabulary))  # If term not present, assign next int.
    for tokens in tokens_list:
        for token in tokens:
            vocabulary[token]  # looking up a key; defaultdict takes care of assigning it a value.
    print('%d unique terms in vocabulary' % len(vocabulary))
    return vocabulary

In [61]:
vocabulary = make_vocabulary(tokens_list)

20542 unique terms in vocabulary


In [62]:
# term->index
list(vocabulary.items())[:10]

[('THIS_IS_A_MENTION', 0),
 ('happy', 1),
 ('birthday', 2),
 ('d=fun', 3),
 ('stuart', 4),
 ('fl', 5),
 ('was', 6),
 ('fun', 7),
 ('jensen', 8),
 ('beach', 9)]

In [63]:
# How big is vocabulary if we keep punctuation?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=True, collapse_mentions=True)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

29796 unique terms in vocabulary


In [64]:
# How big is vocabulary if we keep punctuation and urls?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=True)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

32465 unique terms in vocabulary


In [65]:
# How big is vocabulary if we keep punctuation and urls and mentions?
tokens_list = [tweet2tokens(t, use_descr=True, lowercase=True,
                            keep_punctuation=True, descr_prefix='d=',
                            collapse_urls=False, collapse_mentions=False)
              for t in tweets]

vocabulary = make_vocabulary(tokens_list)

35782 unique terms in vocabulary


## Feature Vector Matrix

Create a matrix $X$ where $X[i,j]$ is the frequency of term $j$ in tweet $i$.

$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\end{pmatrix}
$$



## Sparse Matrices

$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\end{pmatrix}
$$

$X$ is mostly $0$ for text problems.

## List of List (LIL) Matrix

Store a linked list of (index, value) pairs for each row.

$$
X = \begin{pmatrix}
\hbox{tweet}_1 & (0, 1)\\
\hbox{tweet}_2 & (3,2)\\
\hbox{tweet}_3 & (0,1), (1,1)\\
\end{pmatrix}
$$

**Advantage:** Fast to construct: append to list in constant time.

**Disadvantage:** Slow random access for matrix-vector product.

E.g., $\hat{z} = X\cdot \hat{\beta}$ to classify tweets using a learned weight vector $\beta$

$\hat{z}[i] = \sum_j X[i,j] * \beta[j]$

## Compressed Sparse Row (CSR) Matrix


$$
X = \begin{pmatrix}
~ & \hbox{term}_1 & \hbox{term}_2 & \hbox{term}_3 & \hbox{term}_4 \\
\hbox{tweet}_1 & 1  &  0  &  0 & 0 \\
\hbox{tweet}_2 & 0  &  0  &  0 & 2 \\
\hbox{tweet}_3 & 1  &  1  &  0 & 0 \\
\hbox{tweet}_4 & 1  &  0  &  0 & 0 \\
\end{pmatrix}
$$

CSR Matrix is an object with three attributes: 
- **val:** $\{1,2,1,1,1\}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *list of all non-zero values*  
- **col_ind:** $\{0,3,0,1,0\}$ &nbsp; *column index for each non-zero value* (e.g., first non-zero value (1) is in column 0) 
- **row_ptr:** $\{0,1,2,4\}$ &nbsp;&nbsp;&nbsp; *index into **col_ind** where each row starts* (e.g., tweet3, term1 corresponds to col_ind[2])

Allows efficient row access (good for us, since each row is a tweet)

In [22]:
# Convert features to a sparse matrix X.
# X[i,j] is the frequency of term j in tweet i
# 
from scipy.sparse import lil_matrix

def make_feature_matrix(tokens_list, vocabulary):
    X = lil_matrix((len(tweets), len(vocabulary)))
    for i, tokens in enumerate(tokens_list):
        for token in tokens:
            j = vocabulary[token]
            X[i,j] += 1
    return X.tocsr()  # convert to CSR for more efficient random access.

In [66]:
X = make_feature_matrix(tokens_list, vocabulary)
print('shape of X:', X.shape)

shape of X: (5000, 35782)


In [67]:
help(X)

Help on csr_matrix in module scipy.sparse.csr object:

class csr_matrix(scipy.sparse.compressed._cs_matrix, scipy.sparse.sputils.IndexMixin)
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_matrix(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
 |          corresponding value

In [68]:
# How is tweet 200 stored?
X[200]

<1x35782 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [26]:
help(X[200].nonzero)

Help on method nonzero in module scipy.sparse.base:

nonzero() method of scipy.sparse.csr.csr_matrix instance
    nonzero indices
    
    Returns a tuple of arrays (row,col) containing the indices
    of the non-zero elements of the matrix.
    
    Examples
    --------
    >>> from scipy.sparse import csr_matrix
    >>> A = csr_matrix([[1,2,0],[0,0,3],[4,0,5]])
    >>> A.nonzero()
    (array([0, 0, 1, 2, 2]), array([0, 1, 2, 0, 2]))



In [70]:
X[200].nonzero()

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=int32),
 array([  29,   42,   56,   61,   63,  101,  102,  112,  269,  437,  494,
         836,  936, 1082, 1368, 1374, 2431, 2788, 2789, 2790, 2791, 2792,
        2793, 2794, 2795, 2796, 2797, 2798, 2799], dtype=int32))

In [76]:
# non-zero indices of terms used in tweet 200.
X[200].nonzero()[1]

array([  29,   42,   56,   61,   63,  101,  102,  112,  269,  437,  494,
        836,  936, 1082, 1368, 1374, 2431, 2788, 2789, 2790, 2791, 2792,
       2793, 2794, 2795, 2796, 2797, 2798, 2799], dtype=int32)

In [72]:
# term counts for tweet 200.
X[200].data

array([ 2.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.])

In [73]:
# What word does each term index correspond to?
# Convert term->index dict into index->term dict
index2term = {i: t for t, i in vocabulary.items()}
print(index2term[29])
print(X[200, 29])
# So, the term "for" (index 29) appears in user 200's tweet two times

for
2.0


In [74]:
# d=and appears one time.
print(index2term[42])
print(X[200, 42])

d=and
1.0


How do CSR matrices access row values?

Recall:

CSR Matrix is an object with three attributes: 
- **val:** $\{1,2,1,1\}$  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *list of all non-zero values*  
- **col_ind:** $\{0,3,0,1\}$ &nbsp; *column index for each non-zero value* (e.g., first non-zero value (1) is in column 0) 
- **row_ptr (ind_ptr):** $\{0,1,2\}$ &nbsp;&nbsp;&nbsp; *index into **col_ind** where each row starts* (e.g., tweet3, term1 corresponds to col_ind[2])

In [31]:
# Recall: numpy array slices.
import numpy as np
a = np.array([0, 100, 200, 300, 400, 500])
a[2:5]  # get elements at positions 2,3,4

array([200, 300, 400])

In [32]:
print('tweet 200 starts at col_ind=', X.indptr[200])
print('tweet 201 starts at col_ind=', X.indptr[201])
print('so, the columns that are non-zero for tweet 200 are:')
print(X.indices[X.indptr[200]:X.indptr[201]])
print('and the data associated with those cells are:')
print(X.data[X.indptr[200]:X.indptr[201]])

tweet 200 starts at col_ind= 4295
tweet 201 starts at col_ind= 4324
so, the columns that are non-zero for tweet 200 are:
[  29   42   56   61   63  101  102  112  269  437  494  836  936 1082 1368
 1374 2431 2788 2789 2790 2791 2792 2793 2794 2795 2796 2797 2798 2799]
and the data associated with those cells are:
[ 2.  1.  1.  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [77]:
print('tweet 0:\n', X[0], '\n')
print('tweet 1:\n', X[1], '\n')
print('tweet 2:\n', X[2])

tweet 0:
   (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0 

tweet 1:
   (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 14)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0 

tweet 2:
   (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 26)	2.0
  (0, 27)	1.0
  (0, 28)	1.0
  (0, 29)	1.0
  (0, 30)	1.0
  (0, 31)	1.0
  (0, 32)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 35)	1.0
  (0, 36)	1.0
  (0, 37)	1.0
  (0, 38)	3.0
  (0, 39)	1.0
  (0, 40)	1.0
  (0, 41)	1.0
  (0, 42)	3.0
  (0, 43)	1.0
  (0, 44)	1.0
  (0, 45)	1.0
  (0, 46)	1.0
  (0, 47)	1.0
  (0, 48)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 51)	2.0
  (0, 52)	1.0
  (0, 53)	1.0
  (0, 54)	1.0
  (0, 55)	1.0
  (0, 56)	1.0
  (0, 57)	1.0
  (0, 58)	1.0
  (0, 59)	1.0
  (0, 60)	1.0


**Efficient matrix vector product:**

$$ z = X * \beta $$

In [34]:
# Compute z = X * \beta, where X is a CSR matrix.
import numpy as np
beta = np.ones(len(vocabulary))  # assume Beta = vector of 1s
z = np.zeros(len(tweets))
for i in range(len(tweets)):  # for each row.
    for j in range(X.indptr[i], X.indptr[i+1]): # for each col.
        colidx = X.indices[j]
        z[i] += beta[colidx] * X.data[j]
print('X * beta for tweet 200=', z[200])
print('which is the same as the sum %.1f, since beta=[1...1]' %
      X[200].sum())

X * beta for tweet 200= 31.0
which is the same as the sum 31.0, since beta=[1...1]


**4.) Create a list of gender labels.**

In [78]:
# y is a 1d numpy array of gender labels.
# Let 1=Female, 0=Male.
import numpy as np

def get_gender(tweet, male_names, female_names):
    name = get_first_name(tweet)
    if name in female_names:
        return 1
    elif name in male_names:
        return 0
    else:
        return -1
    
y = np.array([get_gender(t, male_names, female_names) for t in tweets])
print('gender labels:', Counter(y))

gender labels: Counter({0: 2693, 1: 2307})


**5.) Fit a Logistic Regression classifier to predict gender from profile/tweet.**

#Sports Guy

In [83]:
# Do 5-fold cross-validation
# http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

def do_cross_val(X, y, nfolds):
    """ Compute average cross-validation acccuracy."""
    cv = KFold(n_splits=nfolds, random_state=42, shuffle=True)
    accuracies = []
    for train_idx, test_idx in cv.split(X):
        clf = LogisticRegression()
        clf.fit(X[train_idx], y[train_idx])
        predicted = clf.predict(X[test_idx])
        acc = accuracy_score(y[test_idx], predicted)
        accuracies.append(acc)
    avg = np.mean(accuracies)
    print(np.std(accuracies))
    print(accuracies)
    return avg

In [84]:
print('avg accuracy', do_cross_val(X, y, 5))

0.0171417618698
[0.747, 0.70599999999999996, 0.71599999999999997, 0.71299999999999997, 0.69599999999999995]
avg accuracy 0.7156


In [38]:
# Fitting model with CSR much, much faster than with LIL.
from timeit import timeit
print('CSR TIME')
timeit("do_cross_val(X.tocsr(), y, 2)", number=5,
       setup="from __main__ import do_cross_val, X, y")

CSR TIME


0.5853505079867318

In [122]:
print('LIL TIME')
timeit("do_cross_val(X.tolil(), y, 2)", number=5,
       setup="from __main__ import do_cross_val, X, y")

LIL TIME


214.12728118896484

(See more about vectorization of arithmetic operations: 
https://en.wikipedia.org/wiki/Automatic_vectorization )

In [39]:
# How does tokenization affect accuracy?
# Collapse urls and mentions; ignore description prefix.
def run_all(tweets, use_descr=True, lowercase=True,
            keep_punctuation=True, descr_prefix=None,
            collapse_urls=True, collapse_mentions=True):
    
    tokens_list = [tweet2tokens(t, use_descr, lowercase,
                            keep_punctuation, descr_prefix,
                            collapse_urls, collapse_mentions)
                  for t in tweets]
    vocabulary = make_vocabulary(tokens_list)
    X = make_feature_matrix(tokens_list, vocabulary)
    acc = do_cross_val(X, y, 5)
    print('acc=', acc)
    return acc

In [40]:
argnames = ['use_descr', 'lower', 'punct', 'prefix', 'url', 'mention']
option_iter = product(use_descr_opts, lowercase_opts,
                       keep_punctuation_opts,
                       descr_prefix_opts, url_opts,
                       mention_opts)
results = []
for options in option_iter:
    option_str = '\t'.join('%s=%s' % (name, opt) for name, opt
                           in zip(argnames, options))
    print(option_str)
    acc = run_all(tweets, *options)
    results.append((acc, options))
    print

use_descr=True	lower=True	punct=True	prefix=d=	url=True	mention=True
29796 unique terms in vocabulary
acc= 0.7114
use_descr=True	lower=True	punct=True	prefix=d=	url=True	mention=False
33113 unique terms in vocabulary
acc= 0.7148
use_descr=True	lower=True	punct=True	prefix=d=	url=False	mention=True
32465 unique terms in vocabulary
acc= 0.7114
use_descr=True	lower=True	punct=True	prefix=d=	url=False	mention=False
35782 unique terms in vocabulary
acc= 0.7156
use_descr=True	lower=True	punct=True	prefix=	url=True	mention=True
26212 unique terms in vocabulary
acc= 0.7072
use_descr=True	lower=True	punct=True	prefix=	url=True	mention=False
29479 unique terms in vocabulary
acc= 0.7116
use_descr=True	lower=True	punct=True	prefix=	url=False	mention=True
28882 unique terms in vocabulary
acc= 0.7076
use_descr=True	lower=True	punct=True	prefix=	url=False	mention=False
32149 unique terms in vocabulary
acc= 0.711
use_descr=True	lower=True	punct=False	prefix=d=	url=True	mention=True
20542 unique terms 

In [41]:
for r in sorted(results, reverse=True):
    print('%.4f' % r[0], '  '.join('%s=%s' % (name, opt) for name, opt in zip(argnames, r[1])))

0.7200 use_descr=True  lower=False  punct=False  prefix=d=  url=False  mention=False
0.7198 use_descr=True  lower=False  punct=False  prefix=d=  url=True  mention=False
0.7178 use_descr=True  lower=False  punct=False  prefix=d=  url=True  mention=True
0.7166 use_descr=True  lower=False  punct=False  prefix=d=  url=False  mention=True
0.7166 use_descr=True  lower=False  punct=False  prefix=  url=False  mention=False
0.7156 use_descr=True  lower=True  punct=True  prefix=d=  url=False  mention=False
0.7148 use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=False
0.7144 use_descr=True  lower=False  punct=False  prefix=  url=True  mention=False
0.7142 use_descr=True  lower=True  punct=False  prefix=d=  url=True  mention=False
0.7116 use_descr=True  lower=True  punct=True  prefix=  url=True  mention=False
0.7114 use_descr=True  lower=True  punct=True  prefix=d=  url=True  mention=True
0.7114 use_descr=True  lower=True  punct=True  prefix=d=  url=False  mention=True
0.7110 u

In [48]:
idx2word = dict((v,k) for k,v in vocabulary.items())

In [85]:
# Fit model on all data and print top coef.
model = LogisticRegression()
model.fit(X,y)
# Get the learned coefficients for the Positive class.
coef = model.coef_[0]
# Sort them in descending order.
top_coef_ind = np.argsort(coef)[::-1][:10]
# Get the names of those features.
top_coef_terms = [idx2word[i] for i in top_coef_ind]
# Get the weights of those features
top_coef = coef[top_coef_ind]
# Print the top 10.
print('top weighted terms for female class:')
print('\n'.join(str(x) for x in zip(top_coef_terms, top_coef)))

# repeat for males
top_coef_ind = np.argsort(coef)[:10]
top_coef_terms = [idx2word[i] for i in top_coef_ind]
top_coef = coef[top_coef_ind]
print('\ntop weighted terms for male class:')
print('\n'.join(str(x) for x in zip(top_coef_terms, top_coef)))


top weighted terms for female class:
('#metoo', 2.3154601392316754)
('d=mom', 1.798766249630277)
('d=girl', 1.7714018994086582)
('d=✨', 1.596262699318054)
('d=mother,', 1.4193849296688583)
('d=mother', 1.374360934314119)
('d=♡', 1.3612915008348505)
('@mainzina', 1.2944598766148903)
('d=mom,', 1.2894364668895928)
('d=:)', 1.1621836076505723)

top weighted terms for male class:
('does', -1.4372816221795532)
('d=sports', -1.382713422889523)
('d=@chefgold1', -1.1799776531858888)
('d=man', -1.1617828042108287)
('d=dad', -1.094014136190202)
('awesome', -1.0682529543259081)
('d=father,', -1.0528898303380099)
('d=husband,', -1.0268638984863108)
('d=founder', -0.99832676951667876)
('d=guy', -0.97336485347454016)


## Error Analysis

- Which ones do we get wrong?
- Are there obvious reasons?